# NLP를 활용한 레시피 추천 챗봇 구현
**개요**
- Collection 
- Preprocessing
- EDA
- Embeddings
- Modeling
- Streamlit

## 1. Collection (웹 크롤링 : 스크래핑)
- 수집할 데이터 : 요리이름, 카테고리, 난이도, 소요시간, 재료, 요리방법, 요리설명, 링크, 이미지링크
- 수집 대상 사이트 : 우리의 식탁(https://wtable.co.kr/recipes)
- 수집 방법 : 웹 크롤링(스크래핑)
    - BeautifulSoup
    - requests

### 라이브러리 import

In [1]:
# 데이터 분석
import numpy as np
import pandas as pd

# 웹 스크래핑
import requests
from bs4 import BeautifulSoup as bs

# HTTP 클라이언트
import urllib3
urllib3.disable_warnings()

# 진행시간 표시
import time
from tqdm.notebook import tqdm
tqdm.pandas()

### 웹 스크래핑 진행하기

In [2]:
def get_link(token):
    "음식 상세정보 링크 반환"
    link = f'https://wtable.co.kr/recipes/{token}?location=recipe_home'
    return link

In [3]:
# 카테고리 코드
category = {'메인요리': 1,
            '초대요리': 2,
            '간식': 5,
            '간단요리': 126,
            '밑반찬': 183,
            '채식': 184,
            '한식': 185,
            '양식': 186,
            '일식': 187,
            '중식': 188,
            '퓨전': 189,
            '분식': 190,
            '안주': 191,
            '베이킹': 192,
            '다이어트': 193,
            '도시락': 214,
            '키토': 242,
            '오븐 요리': 273
            }

In [4]:
def get_one_page(cuisine, start):
    "cuisine의 start번째부터 40개 수집"
    url_base = f'https://wtable.net/api_v2/theme/recipe/list?&platform=web&'
    url_base2 = f'theme_id={category[cuisine]}&offset={start}&limit=40'
    url = url_base + url_base2

    response = requests.get(url, verify=False)
    data = response.json()['data']

    level_list = list(map(lambda x: x['level'], data))
    time_list = list(map(lambda x: x['time'], data))
    title_list = list(map(lambda x: x['title'], data))
    link_list = [get_link(i['token']) for i in data]
    img_list = list(map(lambda x: x['profile_img'], data))


    df = pd.DataFrame({'요리':title_list,                 
                       '종류':cuisine,
                       '난이도':level_list,
                       '소요시간':time_list,
                       '링크':link_list,
                       '사진':img_list})

    return df

In [5]:
def get_all_pages(cuisine):
    "모든 페이지 수집"
    start = 0
    page_list = []
    while True:
        df_one_page = get_one_page(cuisine, start)
        if df_one_page.empty:
            break
        else:
            page_list.append(df_one_page)
            time.sleep(0.01)
            start += 40

    df_all_page = pd.concat(page_list, ignore_index=True)
    
    return df_all_page

In [6]:
# 모든 카테고리를 대상으로 get_all_pages 적용하기
cuisines = []
for i in category:
    cuisines.append(get_all_pages(i))
result = pd.concat(cuisines, ignore_index=True)

In [7]:
def get_details(url):
    "링크에서 재료와 요리방법, 설명 수집 함수"
    try: 
        response = requests.get(url, verify=False)
        soup = bs(response.text)    
        tmp = soup.find_all('div', {'class':'Text__Description02-sc-1qy6bx2-0 fCbbYE'})
        ingredients = [i.find_all('div')[0].text for i in tmp]
        tmp2 = soup.find_all('p', {'class':'Text__Pre01-sc-1qy6bx2-2 enJPxd'})
        cook = [i.text for i in tmp2]
        description = soup.find('p', {'class':'RecipeDetailstyle__Description-q7sykd-14 IdQIJ'}).text
        return ingredients, cook, description
    
    except:
        return None, None, None

In [8]:
# 링크에서 재료와 요리방법, 설명 가져오기
details = result['링크'].progress_map(get_details)

  0%|          | 0/4947 [00:00<?, ?it/s]

In [9]:
result['재료'] = [i[0] for i in details]
result['요리방법'] = [i[1] for i in details]
result['설명'] = [i[2] for i in details]

### 수집한 데이터 확인하기

In [10]:
# 수집한 데이터 구성 요약
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4947 entries, 0 to 4946
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요리      4947 non-null   object
 1   종류      4947 non-null   object
 2   난이도     4947 non-null   int64 
 3   소요시간    4947 non-null   int64 
 4   링크      4947 non-null   object
 5   사진      4947 non-null   object
 6   재료      4947 non-null   object
 7   요리방법    4947 non-null   object
 8   설명      4947 non-null   object
dtypes: int64(2), object(7)
memory usage: 348.0+ KB


In [11]:
# 요리, 종류, 설명만 가져와서 보기
result[['요리', '종류', '설명']]

,요리,종류,설명
0,올리브 소시지 솥밥,메인요리,올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...
1,원 팟 파스타,메인요리,"냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간..."
2,그릴드 브리치즈,메인요리,"브리치즈와 토마토, 오일과 허브를 뿌린 후 오븐에 구워내 보세요. 와인안주로 활용하..."
3,그릴드 피치 샐러드,메인요리,복숭아에 멋진 그릴 자국을 내보세요. 여름을 맞아 제철인 단단한 복숭아를 그릴에 구...
4,과카몰리 부리또콘,메인요리,아보카도는 불포화 지방산이 풍부해 콜레스테롤 수치를 낮추고 혈관건강에 유익해요. 게...
...,...,...,...
4942,골뱅이튀김과 파채,오븐 요리,"특유의 쫄깃한 식감이 매력적인 골뱅이. 지금까지 골뱅이무침만 먹어 오셨다면, 오늘은..."
4943,대하치즈구이,오븐 요리,비주얼부터 감탄에 감탄을 부르는 대하치즈구이! 탱글탱글한 대하를 동그랗게 펼친 후 ...
4944,돈마호크,오븐 요리,"소고기에 토마호크가 있다면, 돼지고기엔 돈마호크가 있죠. 돈마호크는 돼지고기의 삼겹..."
4945,단호박 에그슬럿,오븐 요리,노란빛의 달콤한 속살을 가진 단호박은 식이섬유가 많고 열량이 낮아 다이어트에 효과적...


In [12]:
# 데이터 미리보기
result.head(1)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,"[쌀, 물, 치킨스톡 파우더, 샬롯(또는 양파), 그린올리브, 소시지, 파마산 치즈...","[쌀은 씻어 채에 받쳐 30분 정도 불려주세요. , 소세지는 모양대로 슬라이스 해주...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...


### 수집한 데이터 저장하기

In [13]:
result.to_csv('data/raw_recipes.csv', index=False)

In [14]:
raw_recipes = pd.read_csv('data/raw_recipes.csv')
raw_recipes.head()

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,"['쌀', '물', '치킨스톡 파우더', '샬롯(또는 양파)', '그린올리브', '...","['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...
1,원 팟 파스타,메인요리,1,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,"['파스타면', '방울토마토', '그린 올리브', '케이퍼', '파슬리', '올리브...","['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간..."
2,그릴드 브리치즈,메인요리,1,30,https://wtable.co.kr/recipes/Tzr7zXZuJvm35s1wd...,https://static.wtable.co.kr/image/production/s...,"['브리치즈', '방울토마토', '로즈마리', '마늘', '올리브오일', '후춧가루...","['브리치즈에 칼집을 넣어주세요. 마늘은 편을 썰어주세요.\xa0', '오븐용기에 ...","브리치즈와 토마토, 오일과 허브를 뿌린 후 오븐에 구워내 보세요. 와인안주로 활용하..."
3,그릴드 피치 샐러드,메인요리,1,25,https://wtable.co.kr/recipes/RofraF8aFP5u5rqeX...,https://static.wtable.co.kr/image/production/s...,"['복숭아', '부라타치즈', '루꼴라', '방울토마토', '민트잎', '화이트발사...",['복숭아는 씨를 빼고 웨지 모양으로 썰어주세요. 작은 것은 반으로 썰어 주세요.\...,복숭아에 멋진 그릴 자국을 내보세요. 여름을 맞아 제철인 단단한 복숭아를 그릴에 구...
4,과카몰리 부리또콘,메인요리,1,20,https://wtable.co.kr/recipes/3PW95soZHRArnzsYB...,https://static.wtable.co.kr/image/production/s...,"['토르티야', '아보카도', '토마토', '양파', '레몬', '소금', '고수'...",['또띠아를 반으로 자른 후 원 뿔 모양으로 둥글게 말아 나무 꼬치로 가장자리를 고...,아보카도는 불포화 지방산이 풍부해 콜레스테롤 수치를 낮추고 혈관건강에 유익해요. 게...
